In [ ]:

!pip install pyopencl


In [2]:
import pyopencl as cl
import numpy as np
import pandas as pd
import os

from PIL import Image, ImageFilter

C:\Users\Eevee\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


In [4]:
kernel_filter_color_local="""
__kernel void kernel_filter_color_local(
    __global uchar* imagen_in, 
    __global uchar* imagen_out, 
    __constant float* filtro, 
    int dim, 
    int ancho, 
    int alto, 
    __local uchar* local_imagen) 
{
    int fila = get_global_id(0);
    int columna = get_global_id(1);
    
    int centro = (dim - 1) / 2;

    // IDs locales
    int local_fila = get_local_id(0);
    int local_columna = get_local_id(1);
    
    // Tamaño del grupo de trabajo
    int local_size_x = get_local_size(0);
    int local_size_y = get_local_size(1);
    
    // Índice global
    int global_idx = (fila * ancho + columna) * 3;

    // Índice local en memoria local
    int local_idx = ((local_fila + centro) * (local_size_x + 2 * centro) + (local_columna + centro)) * 3;

    // Cargar píxeles en memoria local desde imagen_in
    int count = 0;
    if (fila < alto && columna < ancho) {
        for (int c = 0; c < 3; c++) {
            local_imagen[local_idx + c] = imagen_in[global_idx + c];
            count++;
        }
    }

    // Cargar bordes
    for (int dy = -centro; dy <= centro; dy++) {
        for (int dx = -centro; dx <= centro; dx++) {
            int local_x = local_columna + dx + centro;
            int local_y = local_fila + dy + centro;
            int global_x = columna + dx;
            int global_y = fila + dy;
            
            if (local_x >= 0 && local_x < local_size_x + 2 * centro && 
                local_y >= 0 && local_y < local_size_y + 2 * centro && 
                global_x >= 0 && global_x < ancho && 
                global_y >= 0 && global_y < alto) 
            {
                int local_mem_idx = (local_y * (local_size_x + 2 * centro) + local_x) * 3;
                int global_mem_idx = (global_y * ancho + global_x) * 3;
                local_imagen[local_mem_idx] = imagen_in[global_mem_idx];
                local_imagen[local_mem_idx + 1] = imagen_in[global_mem_idx + 1];
                local_imagen[local_mem_idx + 2] = imagen_in[global_mem_idx + 2];
            }
        }
    }

    barrier(CLK_LOCAL_MEM_FENCE);

    // Aplicar el filtro
    if (centro <= fila && fila < (alto - centro) && centro <= columna && columna < (ancho - centro)) {
        float suma_rojo = 0.0f;
        float suma_verde = 0.0f;
        float suma_azul = 0.0f;

        for (int i = -centro; i <= centro; i++) {
            for (int j = -centro; j <= centro; j++) {
                int local_idx = ((local_fila + i + centro) * (local_size_x + 2 * centro) + (local_columna + j + centro)) * 3;

                suma_rojo += local_imagen[local_idx] * filtro[(i + centro) * dim + (j + centro)];
                suma_verde += local_imagen[local_idx + 1] * filtro[(i + centro) * dim + (j + centro)];
                suma_azul += local_imagen[local_idx + 2] * filtro[(i + centro) * dim + (j + centro)];
            }
        }

        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = (uchar)clamp(suma_rojo, 0.0f, 255.0f);
        imagen_out[idx_out + 1] = (uchar)clamp(suma_verde, 0.0f, 255.0f);
        imagen_out[idx_out + 2] = (uchar)clamp(suma_azul, 0.0f, 255.0f);
    } else {
        // Manejo de bordes
        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = imagen_in[idx_out];
        imagen_out[idx_out + 1] = imagen_in[idx_out + 1];
        imagen_out[idx_out + 2] = imagen_in[idx_out + 2];
    }
}

"""

In [146]:

kernel_filter_color_local2 = """
__kernel void kernel_filter_color_local2(
    __global uchar* imagen_in, 
    __global uchar* imagen_out, 
    __constant float* filtro, 
    int dim, 
    int ancho, 
    int alto, 
    __local uchar* local_imagen) 
{   
    // Posición pixel global
    int fila = get_global_id(0);
    int columna = get_global_id(1);
    
    int centro = (dim - 1) / 2;

    // IDs locales
    int local_fila = get_local_id(0);
    int local_columna = get_local_id(1);
    
    // Tamaño del grupo de trabajo
    int local_size_x = get_local_size(0);
    int local_size_y = get_local_size(1);
    
    // Índice global
    int global_idx = (fila * ancho + columna) * 3;

    // Variables auxiliares
    int i, j;

    // Acceder a la memoria local solo la hebra 0
    if(local_fila == 0 && local_columna == 0) {
        // Quiero guardar una matriz de tamaño (local_size_x + 2) * (local_size_y + 2) * 3

        for(i = -centro; i < centro+local_size_x; i++) {
            for(j = -centro; j < centro+local_size_y; j++) {
                int fila_actual = fila + i;
                int columna_actual = columna + j;

                if((fila_actual >= 0) && (columna_actual >= 0) && (fila_actual < alto) && (columna_actual < ancho)) {
                    // Índice local en memoria local
                    int local_idx = ((i + centro) * (local_size_y + 2) + (j + centro)) * 3;

                    // Guardar en memoria local
                    for (int c = 0; c < 3; c++) {
                        local_imagen[local_idx + c] = imagen_in[(fila_actual * ancho + columna_actual) * 3 + c];
                    }  
                } else {
                    // Inicializar en caso de estar fuera de los límites
                    int local_idx = (( i + centro) * (local_size_y + 2) + (local_columna + j + centro)) * 3;
                    for (int c = 0; c < 3; c++) {
                        local_imagen[local_idx + c] = 0; // Valor por defecto
                    }
                }
            }
        }
        // Cargar bordes
   
    }

    barrier(CLK_LOCAL_MEM_FENCE);

    // Aplicar el filtro
    if (centro <= fila && fila < (alto - centro) && centro <= columna && columna < (ancho - centro)) {
        float suma_rojo = 0.0f;
        float suma_verde = 0.0f;
        float suma_azul = 0.0f;

        for (int i = -centro; i <= centro; i++) {
            for (int j = -centro; j <= centro; j++) {
                int local_idx = ((local_fila + i + centro) * (local_size_y + 2) + (local_columna + j + centro)) * 3;

                suma_rojo += local_imagen[local_idx] * filtro[(i + centro) * dim + (j + centro)];
                suma_verde += local_imagen[local_idx + 1] * filtro[(i + centro) * dim + (j + centro)];
                suma_azul += local_imagen[local_idx + 2] * filtro[(i + centro) * dim + (j + centro)];
            }
        }

        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = (uchar)clamp(suma_rojo, 0.0f, 255.0f);
        imagen_out[idx_out + 1] = (uchar)clamp(suma_verde, 0.0f, 255.0f);
        imagen_out[idx_out + 2] = (uchar)clamp(suma_azul, 0.0f, 255.0f);
    } else {
        // Manejo de bordes
        int idx_out = (fila * ancho + columna) * 3;
        imagen_out[idx_out] = imagen_in[idx_out];
        imagen_out[idx_out + 1] = imagen_in[idx_out + 1];
        imagen_out[idx_out + 2] = imagen_in[idx_out + 2];
    }
}
"""

            
              
        

In [50]:
def importar_imagen(path):
  imagen = Image.open(path)
  return imagen

In [52]:
def aplicar_filtro_color_local(image_path, filtro, kernel_code, kernel_name, device_type, local_size):
    # Importar imagen
    imagen = importar_imagen(image_path)

    # Convertirla a un array de tres canales
    imagen_np = np.array(imagen).astype(np.uint8)

    # Dimensiones de la imagen
    tam_x, tam_y, _ = imagen_np.shape
    print(tam_x,tam_y)

    # Crear array para la imagen final
    imagen_out_np = np.empty_like(imagen_np)

    # Plataforma y dispositivo
    platform = cl.get_platforms()[0]
    device = platform.get_devices(device_type=device_type)[0]

    # Crear contexto y cola de comandos
    context = cl.Context([device])
    command_queue = cl.CommandQueue(context, device=device, properties=cl.command_queue_properties.PROFILING_ENABLE)

    # Crear el programa y compilarlo
    program = cl.Program(context, kernel_code).build()

    # Crear el kernel
    kernel_filter = cl.Kernel(program, kernel_name)

    # Obtener el tamaño máximo del grupo de trabajo
    wg_size = kernel_filter.get_work_group_info(cl.kernel_work_group_info.WORK_GROUP_SIZE, device)
    print(f"Tamaño máximo del grupo de trabajo: {wg_size}")

    # Crear buffers de imagen
    buffer_in = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=imagen_np)
    buffer_out = cl.Buffer(context, cl.mem_flags.WRITE_ONLY, imagen_out_np.nbytes)

    # Crear buffer para el filtro
    filtro_buf = cl.Buffer(context, cl.mem_flags.READ_ONLY | cl.mem_flags.COPY_HOST_PTR, hostbuf=filtro)

    #Asignar memoria local
    dim=filtro.shape[0]
    centro = (dim - 1) // 2
    local_size_x, local_size_y = local_size
    local_mem_size = (local_size_x + 2 * centro) * (local_size_y + 2 * centro) * 3  # 3 es por los canales R, G, B
    print(local_mem_size)

    # Define la memoria local
    local_mem = cl.LocalMemory(local_mem_size)

    # Argumentos del kernel
    kernel_filter.set_arg(0, buffer_in)
    kernel_filter.set_arg(1, buffer_out)
    kernel_filter.set_arg(2, filtro_buf)
    kernel_filter.set_arg(3, np.int32(filtro.shape[0]))
    kernel_filter.set_arg(4, np.int32(imagen_np.shape[1]))
    kernel_filter.set_arg(5, np.int32(imagen_np.shape[0]))
    kernel_filter.set_arg(6, local_mem)

    # Ejecutar el kernel
    global_size = (tam_x, tam_y)
    event = cl.enqueue_nd_range_kernel(command_queue, kernel_filter, global_size, local_size)
    event.wait()

    # Leer el buffer de salida
    cl.enqueue_copy(command_queue, imagen_out_np, buffer_out)

    # Obtener el tiempo de ejecución
    exec_time = 1e-9 * (event.profile.end - event.profile.start)

    # Guardar y mostrar la imagen resultante
    imagen_resultante = Image.fromarray(imagen_out_np)
    imagen_resultante.show()
    #imagen_resultante.save("C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/pirineos_filtro_color_local.png")

    return imagen_resultante, exec_time


In [54]:
#SUAVIZAR LA IMAGEN
filtro_mean=np.array([
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9],
    [1/9, 1/9, 1/9]
], dtype=np.float32)

#SUAVIZAR IMAGEN,ELIMINA LOS VALORES DE ALTAS FRECUENCIAS
filtro_gaussiani=np.array([
    [1/16, 2/16, 1/16],
    [2/16, 4/16, 2/16],
    [1/16, 2/16, 1/16]
], dtype=np.float32)

#ENFOCA
filtro_enfoque=np.array([
    [0, -1, 0],
    [-1, 5, 1],
    [0, -1, 0]
], dtype=np.float32)

#DESENFOCA
filtro_desenfoque=np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
], dtype=np.float32)

#FILTRO SOBEL: DETECTA BORDES

filtro_sobel_X=np.array([
    [1, 0, -1],
    [2, 0, -2],
    [1, 0, -1]
], dtype=np.float32)

filtro_sobel_Y=np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
], dtype=np.float32)


In [ ]:

def filtro_medio_local():
    device_type = cl.device_type.CPU
    image_path = "/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/IMAGENES/pirineos_imagen.png"

    # Definir las combinaciones de local_size
    local_sizes = [(1, 1), (2, 2), (4, 4), (8, 8), (16, 16), (16, 8), (8, 16)]

    # Crear columnas para el DataFrame
    columns = [f"({ls[0]}/{ls[1]})" for ls in local_sizes]
    results_df = pd.DataFrame(index=['Imagen Pirineos'], columns=columns)

    # Probar cada local_size y medir el tiempo de ejecución
    for local_size in local_sizes:
        imagen_resultante, exec_time = aplicar_filtro_color_local(image_path, filtro_mean, kernel_filter_color_local, "kernel_filter_color_local", device_type, local_size)

        # Asignar el tiempo de ejecución al DataFrame
        results_df.loc['Imagen Pirineos', f"({local_size[0]}/{local_size[1]})"] = exec_time if exec_time is not None else "NP"
        print(f"local_size: {local_size}, tiempo de ejecución: {exec_time}")

    # Guardar los resultados en un archivo CSV
    results_df.to_csv('/content/drive/My Drive/Colab Notebooks/TFG_OPENCL/FILTROS IMAGENES/Filtros_mean_local_CPU.csv', index=True)

filtro_medio_local()

In [ ]:
PYOPENCL_COMPILER_OUTPUT=1
# Lista de rutas de imágenes
lista_paths = [
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen64x64.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen128x128.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen640x480.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen800x600.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen720x1280.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen1920x1080.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen2160x3840.jpg",
    "C:/Users/Eevee/Documents/OPENCL_TFG/FILTROS IMAGENES/IMAGENES/imagen8000x6000.jpg"
]

device_type = cl.device_type.GPU  
local_sizes = [(1, 1), (2, 2), (4, 4), (8, 8), (16, 16)]
results = {size: [] for size in local_sizes}

# Procesar cada imagen
for path in lista_paths:
    print(f"Processing: {os.path.basename(path)}")
    
    for local_size in local_sizes:
        try:
            imagen_resultante, exec_time = aplicar_filtro_color_local(path, filtro_enfoque, kernel_filter_color_local2, "kernel_filter_color_local2", device_type, local_size)
            results[local_size].append(exec_time)
        except Exception as e:
            print(f"Error processing {os.path.basename(path)} with local_size {local_size}: {e}")
            results[local_size].append(None)  # O puedes agregar un valor especial para indicar error

# Crear DataFrame
df_results = pd.DataFrame(results, index=[os.path.basename(path) for path in lista_paths])

# Agregar la columna con los nombres de las imágenes
df_results.index.name = 'Image Name'

# Mostrar resultados
print(df_results)

In [34]:
device_type = cl.device_type.GPU 
platform = cl.get_platforms()[0]
device = platform.get_devices(device_type=device_type)[0]
max_local_mem = device.get_info(cl.device_info.LOCAL_MEM_SIZE)
print(max_local_mem)

49152
